<h1 dir=rtl align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
    SharifML Contest - Vision
</font>
</h1>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
   
</font>
</p>

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معرفی مجموعه تصاویر
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    در این سوال به‌طور کلی سه تا پوشه وجود دارد:
    <br>
    تصاویر مربوط به تفنگ‌های کوشا در پوشه <code>gun</code> قرار دارند.
    <br>
    تصاویر مربوط به "زبل خان" در پوشه <code>zebel</code> قرار دارند.
    <br>
    تصاویر مربوط به حیوانات در پوشه <code>animals</code> قرار دارند.
    <br>
    تصاویر مربوط به مجوعه‌داده آزمون در پوشه <code>test_images</code> قرار دارند.
</font>
</p>

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
ساختن مجموعه تصاویر
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
در ابتدا نیاز است که با استفاده از تصاویری که کوشا در اختیارتان گذاشته، یک مجموعه‌داده با شرایط گفته‌شده تولید کنید.
</font>
</p>

In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import gc
import random
import math

device = 'cuda' if torch.cuda.is_available() else 'cpu'
%matplotlib inline

In [154]:
def calculate_angle(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    
    angle_radians = math.atan2(dy, dx)
    angle_degrees = math.degrees(angle_radians)
    if angle_degrees > 90:
        angle_degrees -= 180 
    elif angle_degrees < -90:
        angle_degrees += 180
    return angle_degrees


def calculate_angle_np(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    angle_radians = np.arctan2(dy, dx)

    angle_degrees = np.degrees(angle_radians)

    angle_degrees[angle_degrees > 90] -= 180
    angle_degrees[angle_degrees < -90] += 180

    return angle_degrees

In [155]:
num_images = 10000
result_folder = 'train_images'

# num_images = 1000
# result_folder = 'val_images'

for sample in range(num_images):
    canvas = Image.new('RGBA', (500, 300), (255, 255, 255, 255))

    image_paths = ['./gun/' + np.random.choice(os.listdir('gun'))] + [
        './zebel/' + np.random.choice(os.listdir('zebel'))] + ['./animal/' + np.random.choice(os.listdir('animal')) for
                                                               i in range(random.randint(0, 4))]
    images = [Image.open(path).convert('RGBA') for path in image_paths]
    for i in range(len(images)):
        max_edge = max(images[i].width, images[i].height)
        scale_factor = random.randint(80, 120) / max_edge

        new_size = (
            int(images[i].width * scale_factor),
            int(images[i].height * scale_factor)
        )

        images[i] = images[i].resize(new_size, Image.Resampling.LANCZOS)

    positions = []
    for i in range(len(images)):
        x = np.random.randint(250, 500 - images[i].width) if i != 0 else 0
        y = np.random.randint(0, 300 - images[i].height)
        positions.append((x, y))

    for i in np.random.permutation(range(len(images))):
        img = images[i]
        canvas.paste(img, positions[i], img)

    x1 = positions[0][0] + images[0].width / 2
    y1 = positions[0][1] + images[0].height / 2
    x2 = positions[1][0] + images[1].width / 2
    y2 = positions[1][1] + images[1].height / 2
    label = calculate_angle(x1, y1, x2, y2)
    canvas.save(f'{result_folder}/{sample}_{label}_{x1}_{y1}_{x2}_{y2}_.png')

KeyboardInterrupt: 

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
آموزش مدل
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    حال که مجموعه تصاویر را ساخته‌اید، وقت آن است که مدلی آموزش دهید که بتواند متغیر هدف این مسئله را پیش‌بینی کند.
</font>
</p>

In [156]:
class ImageDataset(Dataset):
    def __init__(self, image_dir):
        self.image_dir = image_dir
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)

        image = Image.open(img_path).convert('RGB')

        args = img_name.split('_')
        _, label_str, x1_str, y1_str, x2_str, y2_str, _ = args
        label, x1, y1, x2, y2 = float(label_str), float(x1_str), float(y1_str), float(x2_str), float(y2_str)

        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])
        image = transform(image)
        label = torch.tensor(label)
        positions = torch.tensor([x1, y1, x2, y2])

        return image, label, positions


train_dataset = ImageDataset('train_images')
val_dataset = ImageDataset('val_images')

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [157]:
gc.collect()
torch.cuda.empty_cache()

In [147]:
from torchvision import models

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 1)
model = model.to(device)

C:\Users\maabb\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\maabb\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [148]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.8)
criterion = nn.MSELoss()
num_epochs = 30

In [149]:
for epoch in range(num_epochs):
    epoch_losses = []
    model.train()

    for image, label, _ in train_dataloader:
        image = image.to(device)
        label = label.to(device).unsqueeze(-1)

        pred = model(image)
        optimizer.zero_grad()

        loss = criterion(pred, label)

        loss.backward()
        optimizer.step()

        epoch_losses.append(loss.item())

    val_losses = []
    model.eval()
    with torch.no_grad():
        for image, label, _ in val_dataloader:
            image = image.to(device)
            label = label.to(device).unsqueeze(-1)

            pred = model(image)

            loss = criterion(pred, label)

            val_losses.append(loss.item())

    scheduler.step()
    mean_loss = np.mean(epoch_losses)
    val_mean_loss = np.mean(val_losses)
    print(f"EPOCH#{epoch:2d},\t Train_Loss:{mean_loss:8.6f} \t Val_Loss:{val_mean_loss:8.6f}")

EPOCH# 0,	 Train_Loss:22.101429 	 Val_Loss:1.371396
EPOCH# 1,	 Train_Loss:8.070050 	 Val_Loss:2.295840
EPOCH# 2,	 Train_Loss:3.343343 	 Val_Loss:0.608417
EPOCH# 3,	 Train_Loss:1.444623 	 Val_Loss:2.956528
EPOCH# 4,	 Train_Loss:0.827421 	 Val_Loss:0.974513
EPOCH# 5,	 Train_Loss:0.534892 	 Val_Loss:0.936114
EPOCH# 6,	 Train_Loss:0.497332 	 Val_Loss:0.179758
EPOCH# 7,	 Train_Loss:0.421224 	 Val_Loss:0.262524
EPOCH# 8,	 Train_Loss:0.429310 	 Val_Loss:0.379329
EPOCH# 9,	 Train_Loss:0.309239 	 Val_Loss:0.296010
EPOCH#10,	 Train_Loss:0.254135 	 Val_Loss:0.142019
EPOCH#11,	 Train_Loss:0.235514 	 Val_Loss:0.374138
EPOCH#12,	 Train_Loss:0.200207 	 Val_Loss:0.241589
EPOCH#13,	 Train_Loss:0.295264 	 Val_Loss:0.401710
EPOCH#14,	 Train_Loss:0.361185 	 Val_Loss:0.401215
EPOCH#15,	 Train_Loss:0.148023 	 Val_Loss:0.349817
EPOCH#16,	 Train_Loss:0.153478 	 Val_Loss:0.204208
EPOCH#17,	 Train_Loss:0.153689 	 Val_Loss:0.143967
EPOCH#18,	 Train_Loss:0.161169 	 Val_Loss:0.218714
EPOCH#19,	 Train_Loss:0.170175

In [150]:
torch.save(model.state_dict(), 'model3.pth')

In [158]:
model2 = models.resnet18(pretrained=True)
model2.fc = nn.Linear(model.fc.in_features, 4)
model2 = model2.to(device)

C:\Users\maabb\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\maabb\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [159]:
optimizer = torch.optim.Adam(model2.parameters(), lr=0.0003)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.8)
criterion = nn.MSELoss()
num_epochs = 5

In [160]:
for epoch in range(num_epochs):
    epoch_losses = []
    model2.train()

    for image, label, positions in train_dataloader:
        image = image.to(device)
        positions = positions.to(device)

        pred = model2(image)
        optimizer.zero_grad()

        loss = criterion(pred, positions / 100)

        loss.backward()
        optimizer.step()

        epoch_losses.append(loss.item())

    val_losses = [0]
    # model2.eval()
    # with torch.no_grad():
    #     for image, label, positions in val_dataloader:
    #         image = image.to(device)
    #         positions = positions.to(device)
    # 
    #         pred = model2(image)
    # 
    #         loss = criterion(pred, positions / 100)
    # 
    #         val_losses.append(loss.item())

    scheduler.step()
    mean_loss = np.mean(epoch_losses)
    val_mean_loss = np.mean(val_losses)
    print(f"EPOCH#{epoch:2d},\t Train_Loss:{mean_loss:8.6f} \t Val_Loss:{val_mean_loss:8.6f}")

EPOCH# 0,	 Train_Loss:0.066297 	 Val_Loss:0.000000
EPOCH# 1,	 Train_Loss:0.002086 	 Val_Loss:0.000000
EPOCH# 2,	 Train_Loss:0.001049 	 Val_Loss:0.000000
EPOCH# 3,	 Train_Loss:0.000767 	 Val_Loss:0.000000
EPOCH# 4,	 Train_Loss:0.000652 	 Val_Loss:0.000000


In [161]:
torch.save(model2.state_dict(), 'model5.pth')

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
معیار ارزیابی
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    معیاری که برای ارزیابی عملکرد مدل انتخاب کرده‌ایم، به‌صورت زیر است.
    <br>
    این معیار، سنجه ارزیابی کیفیت مدل شماست. به عبارت بهتر در سامانه داوری هم از همین معیار برای نمره‌دهی استفاده شده است.
    <br>
    پیشنهاد می‌شود با توجه به این معیار، عملکرد مدل خود را بر روی مجموعه‌ی آموزش یا اعتبارسنجی ارزیابی کنید.
</font>
</p>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font color="red"><b color='red'>توجه:</b></font>
<font face="vazir" size=3>
    برای دریافت نمره از این سوال لازم است تا دقت مدل شما از آستانه‌ی ۰.۴ بیشتر باشد.
    در صورتی که دقت مدل شما از ۰.۴ کمتر باشد نمره شما 
    <b>صفر</b>
    خواهد شد و در غیر این صورت با فرمول زیر محاسبه می‌شود:
</font>
</p>


In [162]:
# Evaluate your model

from sklearn.metrics import make_scorer
import numpy as np


def custom_accuracy(y_true, y_pred):
    """
    Custom accuracy function for scikit-learn.
    Returns the proportion of predictions where the absolute difference 
    between the true and predicted values is <= 0.1.
    """
    # Compute the absolute difference
    abs_diff = np.abs(y_true - y_pred)

    # Check if the difference is <= 0.1
    within_threshold = abs_diff <= 0.1

    # Calculate the proportion of correct predictions
    accuracy = np.mean(within_threshold)
    return accuracy


# Create a scorer for scikit-learn
custom_scorer = make_scorer(custom_accuracy, greater_is_better=True)

<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
 پیش‌بینی بر روی داده‌ی تست و خروجی
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    پیش‌بینی مدل خود بر روی داده‌های آزمون را در یک دیتافریم (<code>dataframe</code>) به فرمت زیر ذخیره کنید.
</font>
</p>


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    توجه داشته باشید که نام دیتافریم باید <code>submission</code> باشد؛ در غیر این‌صورت، سامانه‌ی داوری قادر به ارزیابی خروجی شما نخواهد بود.
    این دیتافریم صرفا شامل ۲ ستون با نام‌های <code>file_name</code> و <code>label</code> است و ۱۰۰۰ سطر دارد.
    <br>
    به ازای هر سطر موجود در مجموعه‌داده‌ی آزمون، باید یک مقدار پیش‌بینی‌شده داشته باشید که مقدار <code>file_name</code> نام تصویر در پوشه‌ی <code>test_images</code> است و مقدار <code>label</code> پیش‌بینی مدل شما برای تصویر است.
    به‌عنوان مثال جدول زیر، ۵ سطر ابتدایی دیتافریم <code>submission</code> را نشان می‌دهد. البته این مقادیر به‌صورت فرضی هستند و در جواب شما، ممکن است متفاوت باشند.
</font>
</p>

<center>
<div align=center 
style="direction: ltr;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    
||<code>file_name</code>|<code>label</code>|
|:----:|:-----:|:-----:|
|0|image_0001.png|0|
|1|image_0002.png|12|
|2|image_0003.png|15|
|3|image_0004.png|9|
|4|image_0005.png|19|

</font>
</div>
</center>

In [163]:
class TestImageDataset(Dataset):
    def __init__(self, image_dir):
        self.image_dir = image_dir
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)

        image = Image.open(img_path).convert('RGB')
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])
        image = transform(image)

        return image, img_name


# Dataset and DataLoader
image_dir = 'test_images'
dataset = TestImageDataset(image_dir)
dataloader = DataLoader(dataset, batch_size=16, shuffle=False)

model.eval()
submission_list = []
with torch.no_grad():
    for images, file_names in dataloader:
        images = images.to(device)
        # model
        # outputs = model(images)
        # predicted_labels1 = outputs.squeeze().cpu().numpy()

        # model2
        outputs = model2(images)
        outputs = outputs.cpu().numpy()
        x1, y1, x2, y2 = outputs[:, 0], outputs[:, 1], outputs[:, 2], outputs[:, 3]
        predicted_labels2 = calculate_angle_np(x1, y1, x2, y2)

        for file_name, label in zip(file_names, predicted_labels2):
            submission_list.append({'file_name': file_name, 'degree': label})

submission = pd.DataFrame(submission_list)
submission.to_csv('submission.csv', index=False)
submission.head()

,file_name,degree
0,image_0001.png,6.222661
1,image_0002.png,16.967422
2,image_0003.png,17.553898
3,image_0004.png,9.381893
4,image_0005.png,19.575699


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
<b>سلول جواب‌ساز</b>
</font>
</h2>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazir" size=3>
    برای ساخته‌شدن فایل <code>result.zip</code> سلول زیر را اجرا کنید. توجه داشته باشید که پیش از اجرای سلول زیر تغییرات اعمال شده در نت‌بوک را ذخیره کرده باشید (<code>ctrl+s</code>) در غیر این صورت، در پایان مسابقه نمره شما به صفر تغییر خواهد کرد.
    <br>
    همچنین اگر از گوگل کولب برای اجرای این فایل نوت‌بوک استفاده می‌کنید، قبل از ارسال فایل <code>result.zip</code>، آخرین نسخه‌ی نوت‌بوک خود را دانلود کرده و داخل فایل ارسالی قرار دهید.
</font>

In [152]:
import zipfile
import os

if not os.path.exists(os.path.join(os.getcwd(), 'SharifML_Contest_Vision.ipynb')):
    %notebook -e SharifML_Contest_Vision.ipynb


def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)


submission.to_csv('submission.csv', index=False)

file_names = ['SharifML_Contest_Vision.ipynb', 'submission.csv']
compress(file_names)

File Paths:
['SharifML_Contest_Vision.ipynb', 'submission.csv']
